In [34]:
import os 
from pathlib import Path
from dataclasses import dataclass
import urllib.request as request
import zipfile
from cnnClassifier import logger
from cnnClassifier.utils.common import get_size
import requests

In [5]:
Path.cwd()

WindowsPath('c:/Users/achow/Documents/FAANG Senior Data Scientist Manager/Projects/Disease-Clsfcn-End-to-End/research')

In [9]:
new_path=Path("..")


In [11]:
%cd {new_path}

c:\Users\achow\Documents\FAANG Senior Data Scientist Manager\Projects\Disease-Clsfcn-End-to-End


C:\Users\achow\AppData\Roaming\Python\Python312\site-packages\IPython\core\magics\osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [12]:
Path.cwd()

WindowsPath('c:/Users/achow/Documents/FAANG Senior Data Scientist Manager/Projects/Disease-Clsfcn-End-to-End')

In [15]:
@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path


In [31]:
from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml, create_directories

In [32]:
class ConfigurationManager:
    def __init__(
            self,
            config_filepath=CONFIG_FILE_PATH,
            params_filepath=PARAMS_FILE_PATH):
        self.config=read_yaml(config_filepath)
        self.params=read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])
    
    def get_data_ingestion_config(self)->DataIngestionConfig:
        config=self.config.data_ingestion
        create_directories([config.root_dir])
        data_ingestion_config=DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir
        )

        return data_ingestion_config

In [35]:
class DataIngestion:
    def __init__(self,config:DataIngestionConfig):
        self.config=config

    def download_file(self):
        local_data_file=Path(self.config.local_data_file)
        if not local_data_file.exists():
            local_data_file.parent.mkdir(parents=True, exist_ok=True)  # Ensure the directory exists
            response = requests.get(self.config.source_URL, stream=True)
            with local_data_file.open("wb") as file:
                file.write(response.content)
            logger.info(f"{local_data_file} downloaded successfully!")
        else:
            logger.info(f"File already exists with size: {self.get_size(local_data_file)}")
    def extract_zip_file(self):
        """
        Extracts the zip file into the data directory.
        """
        local_data_file = Path(self.config.local_data_file)
        unzip_path = Path(self.config.unzip_dir)
        unzip_path.mkdir(parents=True, exist_ok=True)  # Ensure the directory exists

        with zipfile.ZipFile(local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)
        logger.info(f"Extracted {local_data_file} to {unzip_path}")

In [37]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2024-09-02 17:55:46,097: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-09-02 17:55:46,106: INFO: common: yaml file: params.yaml loaded successfully]
[2024-09-02 17:55:46,108: INFO: common: created directory at: artifacts]
[2024-09-02 17:55:46,109: INFO: common: created directory at: artifacts/data_ingestion]
[2024-09-02 17:55:50,686: INFO: 2045371587: artifacts\data_ingestion\data.zip downloaded successfully!]
[2024-09-02 17:55:51,076: INFO: 2045371587: Extracted artifacts\data_ingestion\data.zip to artifacts\data_ingestion]
